In [ ]:
%matplotlib inline

# Determine street unwind order

Created on:  2016-11-15  
Last update: 2016-11-16  
Contact: michael.szell@moovel.com, michael.szell@gmail.com (Michael Szell)

In [ ]:
# preliminaries
from __future__ import unicode_literals
import sys
import csv
import os
import math
import pprint
import requests
import gzip
from collections import defaultdict
import time
import datetime
import numpy as np
from scipy import stats
import pyprind
import itertools
import logging
from ast import literal_eval as make_tuple
from collections import OrderedDict
from retrying import retry

import osmnx as ox
import networkx as nx
import json
from shapely.geometry import mapping, shape, LineString, LinearRing, Polygon, MultiPolygon
import shapely
import shapely.ops as ops
from functools import partial
import pyproj
from scipy import spatial
from haversine import haversine

import pymongo
from pymongo import MongoClient

# plotting stuff
import matplotlib.pyplot as plt

pathdatain = '/Users/szellmi/Documents/lab-mobviz/_playground/berlin-street-wurst/'
pathdataout = '/Users/szellmi/Documents/lab-mobviz/_playground/berlin-street-wurst/'
pp = pprint.PrettyPrinter(indent=4)

# City parameters (generalize later)
cityname = "berlin"
bbox = "52.33812, 13.0884, 52.675499, 13.76134" 
# berlin: 52.33812, 13.0884, 52.675499, 13.76134
# newyork: 40.496044, -74.255735, 40.915256, -73.700272
# stuttgart: 48.46, 8.79, 48.93, 9.50
# amsterdam: 52.3216, 4.7685, 52.4251, 5.0173
with open(pathdatain + cityname +"_streetnames.txt", "r") as ins:
    streetnames = []
    for line in ins:
        streetnames.append(line.rstrip('\n'))

In [ ]:
client = MongoClient()
db = client[cityname+'_raw']
ways = db['ways']
nodes = db['nodes']
db_derived = client[cityname+'_derived']
streets = db_derived['streets']

In [ ]:
def file_len(fname): # http://stackoverflow.com/questions/845058/how-to-get-line-count-cheaply-in-python
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

def assembleStreet(ways, streetname = "Achillesstraße", drawit = False):
    oxelements = []
    elemnodesint = set()
    elemways = []
    elemnodes = []
    nodesdict = {}
    for i,way in enumerate(ways.find({"tags.name":streetname})):
        for n in way["loc"]["nodes"]:
            elemnodesint.add(int(n))
        elemways.append({"id": int(way["_id"]), "nodes":[int(way["loc"]["nodes"][k]) for k in range(len(way["loc"]["nodes"]))], "tags": way["tags"], "type": "way"})

    for n in elemnodesint:
        for nd in nodes.find({"_id": n}):
            elemnodes.append({"id": int(nd["_id"]), "lat": nd["loc"]["coordinates"][1], "lon": nd["loc"]["coordinates"][0], "type": "node"})
            nodesdict[str(int(nd["_id"]))] = {"lat": nd["loc"]["coordinates"][1], "lon": nd["loc"]["coordinates"][0]}

    oxelements = [{"elements": elemnodes + elemways}]

    G = ox.create_graph(oxelements, retain_all=True)
    if drawit:
        fig, ax = ox.plot_graph(G)
    G = nx.Graph(G)
    G = G.to_undirected()
    ccs = list(nx.connected_component_subgraphs(G))

    components = []
    for c in range(len(ccs)):
        deglist = np.array(list((ccs[c].degree_iter())))
        endptindices = np.where(deglist[:, 1] == 1)
        # Look which of the endpoints lies most western, take that as the source for DFS traversal
        west = float('inf')
        source = deglist[0, 0]
        for i in list(endptindices[0]):
            westthis = nodesdict[str(deglist[i, 0])]["lon"]
            if westthis < west:
                source = deglist[i, 0]
                west = westthis

        component = []
        dfsedges = list(nx.dfs_edges(ccs[c], source))
        nend = dfsedges[0][0]
        path = [str(nend)]
        for e in dfsedges:
            if e[0] == nend: # path grows
                path.append(str(e[1]))
            else: # new path
                component.append(path)
                path = [str(e[0]), str(e[1])]
            nend = e[1]    
        component.append(path) # last path
        components.append(component)
        
    output = {"tags":{"name":streetname}, "components":components, "nodes":nodesdict}
    return output

In [ ]:
# test case. To get its output, insert this in the above function after oxelements = ...
oxelements = [{"elements": [{   'id': 1,
                            'lat': 0,
                            'lon': -1,
                            'type': 'node'},
                            {   'id': 2,
                            'lat': 0,
                            'lon': 1,
                            'type': 'node'},
                            {   'id': 3,
                            'lat': -1,
                            'lon': 1,
                            'type': 'node'},
                            {   'id': 4,
                            'lat': -2,
                            'lon': 1,
                            'type': 'node'},
                            {   'id': 5,
                            'lat': 0,
                            'lon': 2,
                            'type': 'node'},
                            {   'id': 6,
                            'lat': -1,
                            'lon': 0,
                            'type': 'node'},
                            {   'id': 7,
                            'lat': 0,
                            'lon': 3,
                            'type': 'node'},
                            {   'id': 8,
                            'lat': -1,
                            'lon': 4,
                            'type': 'node'},
                            {   'id': 91,
                            'nodes': [1, 2, 3],
                            'type': 'way'},
                            {   'id': 92,
                            'nodes': [2, 5],
                            'type': 'way'},
                            {   'id': 93,
                            'nodes': [3, 4],
                            'type': 'way'},
                            {   'id': 94,
                            'nodes': [6, 3],
                            'type': 'way'},
                            {   'id': 95,
                            'nodes': [7, 8],
                            'type': 'way'}
                           ]}]    
nodesdict = {"1":{'lat': 0, 'lon': -1}, 
            "2":{'lat': 0, 'lon': 1}, 
            "3":{'lat': -1, 'lon': 1}, 
            "4":{'lat': -2, 'lon': 1}, 
            "5":{'lat': 0, 'lon': 2}, 
            "6":{'lat': -1, 'lon': 0}, 
            "7":{'lat': 0, 'lon': 3}, 
            "8":{'lat': -1, 'lon': 4}
            }

In [ ]:
streets.drop()  
bar = pyprind.ProgBar(len(streetnames), bar_char='█', update_interval=1)
for i,streetname in enumerate(streetnames):
    try:
        res = assembleStreet(ways, streetname)
        res["_id"] = i+1
        streets.insert_one(res)
    except:
        print(streetname + " could not be inserted.")
    bar.update(item_id = streetname)


In [ ]:
bla = assembleStreet(ways, "Achenseeweg")

In [ ]:
# Tests
bla = ox.osm_net_download(north=48.80393, south=48.80201, east=9.22657, west=9.22126)
pp.pprint(bla)